In [9]:
import matplotlib.pyplot as plt
import numpy as np
import random
import hashlib

In [5]:
def generate_data(n, mean, std):
    """
    Generates an array of n samples from a normal distribution with the given mean and standard deviation.

    Parameters:
    n (int): Number of samples to generate.
    mean (float): Mean of the normal distribution.
    std (float): Standard deviation of the normal distribution.

    Returns:
    np.ndarray: An array of n samples from the normal distribution.
    """
    data = np.random.normal(loc=mean, scale=std, size=n)
    rounded_data = np.round(data).astype(int)  # Rounds the data to the nearest whole number
    return rounded_data.astype(str)

In [29]:
class BasicRappor():

    def __init__(self, k, h, f, p, q):
        assert type(k) == int, 'k error input'
        assert type(h) == int, 'h error input'
        assert f >= 0 and f <1, 'f error input'
        assert q >= 0 and q <1, 'q error input'
        assert p >= 0 and p <1, 'p error input'
        self.k = k
        self.h = h
        self.f = f
        self.p = p
        self.q = q
        self.max_index = k-1
        self.reports = []

    def clear_reports(self):
        self.reports = []

    def return_report(self, v):
        # Step 1: Signal
        B = self.hash_to_bloom_filter(v)

        # Step 2: Permanent randomized response
        B_prime = self.permanent_randomized_response(B)
        # Step 3: Instantaneous randomized response
        S = self.instantaneous_randomized_response(B_prime)

        self.reports.append(S)
        return[S]

    def hash_to_index(self, s):
        # Create a hash object using SHA-256
        hash_object = hashlib.sha256(s.encode())
        
        # Get the hexadecimal representation of the hash
        hex_digest = hash_object.hexdigest()
        
        # Convert the hex digest to an integer
        hash_int = int(hex_digest, 16)
        
        # Normalize the integer to the desired range
        range_size = self.max_index + 1
        normalized_value = (hash_int % range_size)
        
        return normalized_value

    def hash_to_bloom_filter(self, v):
        assert type(v)==str, 'Value v must be a string'
        # Simulate hashing the value v onto a Bloom filter B of size k using h hash functions
        B = [0] * self.k
        for i in range(self.h):
            B[self.hash_to_index(v)] = 1
        return B

    def permanent_randomized_response(self, B):
        k = len(B)
        B_prime = [0] * k
        for i in range(k):
            rand_val = random.random() # Uniform random variable in [0,1]
            if rand_val < 0.5 * self.f: # Checks if <1/2 f
                B_prime[i] = 1
            elif rand_val < self.f: # Checks if 1/2 f < p < f, so probability of f/2
                B_prime[i] = 0
            else:
                B_prime[i] = B[i]
        return B_prime

    def instantaneous_randomized_response(self, B_prime):
        k = len(B_prime)
        S = [0] * k
        for i in range(k):
            if B_prime[i] == 1:
                S[i] = 1 if random.random() < self.q else 0
            else:
                S[i] = 1 if random.random() < self.p else 0
        return S





In [30]:
# Example usage
v = "client_value"  # Client's value
k = 16  # Size of the Bloom filter
h = 1  # Number of hash functions
f = 0.3  # Longitudinal privacy guarantee parameter
p = 0.3  # Probability for instantaneous response when B_prime[i] = 0
q = 0.4  # Probability for instantaneous response when B_prime[i] = 1

rappor = BasicRappor(k, h, f, p, q)


In [33]:
rappor.return_report(v)

[[1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

In [34]:
rappor.reports

[[0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

## Decoding